In [1]:
# 파이썬 리스트에서 파이토치 텐서
a=[1.0,2.0,3.0]

In [2]:
a[0]
a[2]

3.0

In [3]:
import torch
a=torch.ones(3)  #1로 채운 텐서 만들기 1*3
a  

a[1]

tensor(1.)

In [4]:
float(a[1])

1.0

In [5]:
a[2]=2.0
a

tensor([1., 1., 2.])

In [6]:
points= torch.zeros(6) #1*6
points[0]=4.0
points[1]=1.0
points[2]=5.0
points[3]=3.0
points[4]=2.0
points[5]=1.0
points

tensor([4., 1., 5., 3., 2., 1.])

In [7]:
points= torch.tensor([4.0,1.0,5.0,3.0,2.0,1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [8]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [9]:
#2차원 텐서 사용
points=torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points
points.shape

torch.Size([3, 2])

In [10]:
#초기화시켜서 깔금하게 하자 
points=torch.zeros(3,2)

In [11]:
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [12]:
#인덱스 접근 
points=torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points

points[1][0] #5
points[2][1]   #1

points[1]      #5 3 

tensor([5., 3.])

## 텐서 인덱싱

In [13]:
some_list=list(range(6)) #0~5리스트 
some_list[:] #리스트의 모든요소 0~5
some_list[1:6]#1~5
some_list[:4] #0 1 2 3
some_list[:-3] #첫번째요소부터 뒤에서 3번째까지
#some_list[1:] #1~5
some_list[1:4:2]

[1, 3]

In [14]:
points
points[1:] #1인덱스부터 모든인덱스
points[1:,:] #1인덱스부터 모든인덱스
points[1:,0] #1인덱스부터 [첫번째열만]
points[None] #길이가 1인차원추가

points[None].shape


torch.Size([1, 3, 2])

In [15]:
img_t=torch.randn(3,5,5)
img_t.shape  #3 * 5*5
weights=torch.tensor([0.2126,0.7152,0.0722]) #1*3
batch_t=torch.randn(2,3,5,5) #B C W H


In [16]:
img_gray_naive=img_t.mean(-3)  #3개의 채널을 평균값으로 구해 흑백화면으로 변환시킨다
batch_gray_naive=batch_t.mean(-3) #채널 흑백화 
img_gray_naive.shape, batch_gray_naive.shape 

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

### 브로드 케스팅
- 텐서 연신시 길이가 1인 차원을 알아서 늘려주는 방식

In [17]:
un_weight=weights.unsqueeze(-1).unsqueeze(-1) #2번시행

un_weight.shape
img_weights=(img_t*un_weight) #(3*5*5) * (3*1*1) -> 3*5*5
img_weights.shape

batch_t_weight=batch_t*un_weight # 2,3,5,5 * 3*1*1. -> 2,3,5,5
batch_t_weight.shape

img_gray_weights=img_weights.sum(-3) #3,5,5 -> 5,5
img_gray_weights.shape

batch_gray_sum= batch_t_weight.sum(-3)  #2 3 5 5 -> 2 5 5
batch_gray_sum.shape

batch_t_weight.shape, batch_t.shape, un_weight.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [18]:
batch_t.shape

torch.Size([2, 3, 5, 5])

In [19]:
#다른방법으로 표현
img_fancy = torch.einsum('...chw,c->...hw',img_t,weights)
batch_fency=torch.einsum("...bchw,c->...hw",batch_t,weights) 
##하... 이해하는대 오래걸렷다
#1. torch.einsum을보면 ("...")<=모든 앞에잇는것들
#2. torch.einsum을보면 ("...bchw")<=1번째 매개변수의 구조 
#3. torch.einsum을보면 ("...bchw",c)<=2번째 매개변수의 구조를 기준
#4. torch.einsum을보면 ("...bchw",c->...hw)<=기준으로 부터 바꾸고싶은 차원의 형태 설정 
#5.batch_fency=torch.einsum("...bchw,c->...hw",batch_t,weights) einsum으로 더해서 각각의 값을만듬 즉 마지막에 5*5 행열이됨

img_fancy.shape ,batch_fency.shape

(torch.Size([5, 5]), torch.Size([5, 5]))

In [20]:
# 구분하기 너무어렵다 텐서의 이름을 붙힐수 잇어!!
weights_named= torch.tensor([0.2126,0.7152,0.0722],names=["channels"])
weights_named

/var/folders/jj/2hvrxhh50kd174vgps9cn0r00000gn/T/ipykernel_4570/1748489652.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1791.)
  weights_named= torch.tensor([0.2126,0.7152,0.0722],names=["channels"])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [21]:
img_neamed=img_t.refine_names(...,"channels","rows","clomns") # ...은 생략을 의미한다
batch_named=batch_t.refine_names(...,"channels","rows","clomns") ##...은 생략 
print("img name :",img_neamed.shape,img_neamed.names)
print("Batch name:",batch_named.shape,batch_named.names)

img name : torch.Size([3, 5, 5]) ('channels', 'rows', 'clomns')
Batch name: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'clomns')


In [22]:
#빠진 차원을 채우고 존재하는 차원을 올바른 순서로 정렬해주자
weights_al=weights_named.align_as(img_neamed)
weights_al.shape, weights_al.names #img named와 차원을 맟춰주고 이름도 맟춰줌 

(torch.Size([3, 1, 1]), ('channels', 'rows', 'clomns'))

In [23]:
gray_named=(img_neamed * weights_al).sum("channels") #3 55  3 1 1의 두 행열을 neamed기준으로 결합
gray_named.shape , gray_named.names

(torch.Size([5, 5]), ('rows', 'clomns'))

In [24]:
gray_named=(img_neamed * weights_al).sum("channelss") 
#당연하겟지만 이름이 다르면 오류

RuntimeError: Name 'channelss' not found in Tensor['channels', 'rows', 'clomns'].

In [25]:
#이름이 있는 텐서를 함수밖에서 사용하려면 이름을 none으로 설정해 없에면됨
gray_prain=gray_named.rename(None)
gray_prain.shape , gray_prain.names

(torch.Size([5, 5]), (None, None))

In [3]:
import torch

- 파이썬 숫자는 겍채다
- 파이썬에서 리스트는 연속된 겍채의 컬렉션이다
- 파이썬 인터프리터는 최적화를 거치는 컴파일된 코드보다 느리다

In [4]:
torch.float16 # 16비트 소수
torch.float32 # 32비트 소수
torch.float64 #64q비트 소수

torch.int8 #8비트 정수t
torch.uint8 #8비트 부호없는 정수

torch.int16 #16정수
torch.int32  #32저웃
torch.int64 #64정수
torch.bool #불리언
 
##텐서의 기본타입은 32비트 부동소수점 
torch.tensor([2,2]).dtype   

torch.int64

In [28]:
double_points = torch.ones(10,2,dtype=torch.double)
short_points=torch.tensor([[1,2],[3,4]],dtype=torch.short) #int16

In [29]:
short_points.dtype
##타입반환가능
double_points=torch.zeros(10,2).double()
short_points=torch.ones(10,2).short()

In [30]:
#to메소드 활용
#타입변환
double_points=torch.zeros(10,2).to(torch.double)
short_points=torch.ones(10,2).to(torch.short)



In [31]:
##여러 타입의 연산을 할때 가장 큰 타입의 기준으로 연산이됨
point64=torch.rand(5,dtype=torch.double)
point_short= point64.to(torch.short)
point64*point_short  #높은 타입의 연산이 출력된걸 확인할수 있음

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

##  문서참고
- https://pytorch.org/docs/stable/index.html

In [32]:
a=torch.ones(3,2)
print(a)
a_t=torch.transpose(a,0,1) #input  dim 1 dim2 차원을 바꾸고싶은 두차원
print(a_t)
a.shape, a_t.shape

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


(torch.Size([3, 2]), torch.Size([2, 3]))

In [33]:
a=torch.ones(3,2)
a_t=a.transpose(0,1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [34]:
points=torch.tensor([[4.0,1.0,],[5.0,3.0,],[2.0,1.0,]])
points.storage()

/var/folders/jj/2hvrxhh50kd174vgps9cn0r00000gn/T/ipykernel_4570/1249849988.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [35]:
#포인터 식개념으로 접근이됨 따라서 포인터처럼 접근가능
points_storage=points.storage()
points_storage[0]
points_storage[1]
points_storage[5]
#이를보여주는건 실제 저장곤간을 1차원 배열이라는걸 보여준다 

#응용 참조하고잇는 포인터를 이용해서 값바꾸기
points=torch.tensor([[4.0,1.0,],[5.0,3.0,],[2.0,1.0,]])
points_storage=points.storage()
points_storage[0]=15.0
points #해당 주소값을 참조해서 points의 값을 바꿔버림


tensor([[15.,  1.],
        [ 5.,  3.],
        [ 2.,  1.]])

In [36]:
a=torch.ones(3,2)

a.zero_() #a자체를 인플레이스 해버림 즉 a자체를바꿔버림
a
b = torch.zeros_like(a)
print(b)

b=torch.zeros(3,2) #바뀌지않고 뷰로만 
b

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [37]:
## 텐서의 구조 오프셋 스트라이즈 차원
points=torch.tensor([[4.0,1.0,],[5.0,3.0,],[2.0,1.0,]])
seceond_point=points[2]
seceond_point.storage_offset()

seceond_point.size()
seceond_point.shape
seceond_point.storage_offset()  #전체 메모리에서 해당인덱스의 거리 
points.stride() #각 차원에 대해 한 원소에서 다음 원소로 넘어가기 위해 스토리지에서 얼마나 많은 위치를 건너뛰어야 하는지를 나타내는 값
#3행 2열짜리의 행렬은 1행당 2개의 열이잇으니 2 , 열의 간격으로 넘어갈때 간격이 1 즉 (2,1)


(2, 1)

In [38]:
seceond_point=points[1]
seceond_point.size()

seceond_point.storage_offset()
seceond_point.stride()

(1,)

## 주의 할점 !! 
- 세텐서를 선언하고 값을 변경해도 기존 텐서의 값이 변환된다

In [39]:

seceond_point[0]=10.0
points  #포인터로 인해  [1][0] 의 원래 텐서가 변경됨

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

- 오류를 범하기 쉬운방법이므로 서브텐서를 새탠서로 복제하자

In [40]:
points=torch.tensor([[4.0,1.0,],[5.0,3.0,],[2.0,1.0,]])
seceond_point=points[1].clone()
seceond_point[0]=10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

- 복사없이 행렬 전치하기

In [41]:

points=torch.tensor([[4.0,1.0,],[5.0,3.0,],[2.0,1.0,]])
points
points_t= points.t() # points 텐서의 전치(transpose)를 반환
print(points_t)
#전치햇는대 위 두코드가 똑같이 메모리를 할당하고 있을가?
id(points.storage())== id(points_t.storage()) #트루가뜸 #id = 고유한 식별자(주소)를 반환
points.stride(), points_t.stride() #텐서의 t는 view만 바꾸기만할뿐 텐서의 메모리 배열을 바꿔지 않기때문에 저값이 출력된다

tensor([[4., 5., 2.],
        [1., 3., 1.]])


((2, 1), (1, 2))

## 다차원 전치

In [42]:
some_t =torch.ones(3,4,5)
print(some_t,"\n",some_t.shape)
transpose_t =some_t.transpose(0,2) #0번째와 2번째의 차원을 바꿈 --> 5* 4 * 3
print(transpose_t,"\n",transpose_t.shape)
print(f"somt_t->stride= {some_t.stride()}")
print(f"transpose_t->stride= {transpose_t.stride()}") #-> 20 -> 5 ->1   ----> 1 5 20


tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]]) 
 torch.Size([3, 4, 5])
tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]) 
 torch.Size([5, 4, 3])
somt_t->stride= (20, 5, 1)
transpose_t->stride= (1, 5, 20)


In [43]:
points.is_contiguous()
points_t.is_contiguous()


False

In [44]:
#인접하지않는 텐서를 인접한 텐서로 만들자
points=torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points_t=points.t()
points_t
print(points_t.storage())
print(points.stride())

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
(2, 1)


## contiguous
- 연속적인 배열의 메모리 할당이 되엇느냐?

In [45]:
##컨티기어스를 적용하면?
points_cont=points_t.contiguous()
points_cont

points_cont.storage() ,points_cont.stride()
#앞서 말한 문제를 즉 트렌스포즈할때 배열을 바뀌지않앗던 문제를 해결해줫다 열순서로 


( 4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6],
 (3, 1))

## 텐서 디바이스 속성관리
- 텐서를 gpu를 사용하여만들자

In [46]:
#m1기반이라 cuda가 없다 즉 이것은 사용할수없다 코랩으로 돌릴대 이용하자
#point_gpu = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]],device='cuda')
point_gpu = torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]],device='cpu') #지금은 cpu를태우자
#point_gpu=points.to(device='cuda')
point_gpu=points.to(device='cpu')
point_gpu

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

- gpu가 2개이상있으면 번호를 설정해두면됨

In [47]:
#point_gpu=points.to(device='cuda : 0')

In [48]:
point=2*points
#point_gpu=2*points.to(device="cuda")

### 연산과정
- 1. point텐서를 gpu에 복사한다
- 2 . cpu에 새 텐서를 할당한 후 곱셈결과를 저장
- 3. gpu텐서 핸들을 반환

In [49]:
point_gpu= point+4
point_gpu

tensor([[12.,  6.],
        [14., 10.],
        [ 8.,  6.]])

In [50]:
point_cpu= point_gpu.to(device='cpu') #gpu연산을 cpu연산으로 바꿈

In [51]:
#축약 
#point_gpu= poin.cuda()
#point_gpu= poin.cuda(0)
point_cpu=point_gpu.cpu() #to(device="cpu")랑 같은의미

 ### 넘파이와 호완

In [5]:
points=torch.ones(3,4)
points_np= points.numpy() #넘파이로 변경
points_np
# gpu -> cpu로 넘어감 메모리 공간은 동일시됨



array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

- 넘파이에서 토치로

In [53]:
points=torch.from_numpy(points_np) # 넘파이에서 토치로 동일한 공간을 위치한 텐서를 얻게됨

### 텐서 직렬화 
- 텐서 저장 
- 텐서 불어오기

In [54]:
torch.save(points,'./dataset/sub/ourpoints.t')

In [55]:
with open("./dataset/sub/ourpoints2.t",'wb') as f:
    torch.save(points,f)

In [56]:
test=torch.load("./dataset/sub/ourpoints.t")

In [57]:
with open("./dataset/sub/ourpoints2.t",'rb') as f:
    test= torch.load(f)

In [58]:
test

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

### 주의
- 위방법은 파이토치로만 호환이  가능하므로  악이될수도잇음
- 텐서를 호환하도록 저장하는 방법 필요


###  HDF5 라이브러리
- 이식성이 높고 감범위하게 지원되는 다이어리
- 중첩된 키- 값 딕셔너리에 직렬화된 정형 다차원 배열을 표현
- 데이터 디스크가 잇는 상태에서 원하는 요소만 인덱스로 직접 접근가능

In [6]:
import h5py

In [8]:
f=h5py.File("./dataset/sub/ourpoints.hdf5",'w')
dset=f.create_dataset('coords',data=points.numpy())
f.close()
#corrds가 파일의키임


In [15]:
f=h5py.File("./dataset/sub/ourpoints.hdf5",'r')
dset=f['coords']
last_points=dset[-2:] #마지막 2번째항부터 마지막까지출력
last_points

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [18]:
last_points=torch.from_numpy(dset[-2:])
f.close()


ValueError: Dset_id is not a dataset id (dset_id is not a dataset ID)

In [21]:
last_points.dtype

torch.float32

## 연습문제
### 1   list(range(9)) 로부터 텐서를 만들어보자 사이즈, 오프셋, 스트라이드는 몇인가
- a,b = a.view(3,3)텐서를 만들고 view역활은 무엇인가? # a와 b가 같은 공간을 가르키고 있는가
- c= b[1: , 1:] 로 텐서를 만들고 사이즈 오프셋 스트라이드는 얼마일지 계산해보자

### 2.코사인 제곱근 같은 수학연산을 하나골라 동일한 역활을 하는 torch 라이브러리를 찾자
 - 텐서 a에 대해 해당하는 함수를 요소단위로 실행해보자. 오류발생원인
 - 동작시키려면 어떤 연산필요할까ㅣ
 - 해당연산을 추가공간을 사용하지않고 시행하자

In [36]:
import torch
#la=list(range(9))
#la=torch.tensor(la)
a= torch.tensor(list(range(9)))
a

a.size() #([9])
a.storage_offset() #0
a.stride() # (1,)
a.storage()
b=a.view(3,3) #view는 a의 대한 1*9 행렬을 3*3행렬로 변환시켜 보여준다 하지만 실제 메모리는 1*9가 되어잇으므로 view를 잘못사용하면 컨티기어스문제가 나온다
print(id(a.storage())==id(b.storage())) #트루가뜸 #id = 고유한 식별자(주소)를 반환
print(a.data_ptr() == b.data_ptr())  # True를 반환해야 합니다.
#즉 같은공간을 가르킨다
c= b[1:,1:]
c # [4,5][7,8]
c.size() #[2,2]
c.storage_offset() # 4
c.stride()  # (3,1)

False
True


(3, 1)

In [29]:
print(a.is_contiguous())

print(b.is_contiguous())


True
True


In [40]:
#a_2 =torch.square(a)
#a.square_()
# = torch.tensor([4.0, 9.0, 16.0])
a2=torch.sqrt(a)  
#오류원인 
#예전버전 기준으로 long형태의 정수를 float형태의 소수로 반환하려고해서 오류가 난다 이를 해결하기위해서는 float형태로 형변환을 하면된다
a=a.float()
a.sqrt_() #이렇게하면 인플레이스가 되므로 해당 a의 공간을 그대로 가져와서 값만 바꿀수잇다
#앞서말한 오류가 발생한다 
print(a2)  # tensor([2., 3., 4.]) 
print(a)

tensor([0.0000, 1.0000, 1.0905, 1.1472, 1.1892, 1.2228, 1.2510, 1.2754, 1.2968])
tensor([0.0000, 1.0000, 1.0905, 1.1472, 1.1892, 1.2228, 1.2510, 1.2754, 1.2968])
